## for TA: Instructions on how to import packages

- pip install ta
- pip install pytz

In [1]:
# Imports
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import ta
import numpy as np
import pandas as pd
import pytz 
import math
from datetime import datetime, timedelta, timezone
import threading
import time
import random

#testing
import ib_insync
from ib_insync import *

In [2]:
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        
        #used for caching historical data for placing dummy trades outside of market hours
        self.data = [] 
        
    #historical backtest data
    def historicalData(self, reqId, bar):
        #cache historical data
        self.data.append([bar.date, bar.high, bar.low, bar.close])
        bot.on_bar_update(reqId,bar,False)
    
            
    #on historical data end
    def historicalDataEnd(self, reqId, start, end):
        print(reqId)
    
    #get next order id we can use
    def nextValidId(self, nextorderId):
        global orderId
        orderId = nextorderId
            
    def error(self, id, errorCode, errorMsg):
        print("TWS Error Code: ", errorCode)
        print("Message for Error Code: ",errorMsg)
    
    '''
    METHODS FOR REALTIME TRADING
    '''
    # On Realtime Bar after historical data finishes
    def historicalDataUpdate(self, reqId, bar):
        try:
            bot.on_bar_update(reqId,bar,True)
        except Exception as e:
            print(e)

    # Listen for realtime bars
    def realtimeBar(self, reqId, time, open_, high, low, close,volume, wap, count):
        print("Real-time bars during trading hours")
        super().realtimeBar(reqId, time, open_, high, low, close, volume, wap, count)
        try:
            realTimeBar = Bar(open_, low, high, close, volume, time)
            bot.on_bar_update(reqId, realTimeBar,True)
        except Exception as e:
            print(e)

In [3]:
#Bar Object
class Bar:
    open = 0
    low = 0
    high = 0
    close = 0
    volume = 0
    date = datetime.now()
    def __init__(self, open_=None, low=None, high=None, close=None, volume=None, time=None):
        self.open = open_ or 0
        self.low = low or 0
        self.high = high or 0
        self.close = close or 0
        self.volume = volume or 0
        self.date = time or datetime.now()

In [4]:
# Bot Logic
class Bot:
    
    ib = None
    testIb = None
    bars = []
    reqId = 1
    global orderId
    testOrderId = 1 #increment everyday, resets at midnight
    smaPeriod = 50
    symbol = ''
    currentBar = Bar() #initialize first bar to be all 0's
    initialbartime = datetime.now().astimezone(pytz.timezone("America/New_York"))
    
    def __init__(self):
        
        #connect to IB on init
        self.ib = IBApi()
        
        self.ib.connect('127.0.0.1', 7497, 90)
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        
        ib_thread.start()
        time.sleep(1)
        
        # get inputs
        self.symbol = input('Enter the ticker you want to trade: ')
        self.barsize = input('Enter the bar size: ')
        
        minQuantity = ' min'
        if (int(self.barsize) > 1):
            minQuantity = ' mins'
        
        #create contract object
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        self.ib.reqIds(-1)
        
        #request market data
        print("Requesting market data...")
        self.ib.reqHistoricalData(self.reqId, contract, '', '2 D', str(self.barsize)+minQuantity, 'TRADES',1,1,True,[])
        
        '''
        TESTING FOR HISTORICAL DATA
        '''
        # for testing trading strategy outside of market hours
        #testHistData = self.getHistoricalData()
        #self.testTradingStrategy(testHistData)
        
        '''
        TESTING FOR REALTIME DATA
        '''
        # implicitly calls on_bar_update
        self.ib.reqRealTimeBars(self.reqId, contract, 5, "MIDPOINT", True, [])

    
    #listen to socket in seperate thread
    def run_loop(self):
        self.ib.run()
    
    # NOTE: to only be used outside of trading hours - trading strategy logic test
    # should only be used for placing dummy trades
    def testTradingStrategy(self,testDf):
        
        # reverse dataframe to ensure order is earliest to oldest
        testDf = testDf.iloc[::-1]
        testCloses = testDf['Test: Close']
        self.testSMA = ta.trend.sma_indicator(testCloses, 50, True)
        
        #start from the end (earliest) and work our way up(latest)
        print(testDf.head())
        
        lastLow = testDf.iloc[0,1]
        lastHigh = testDf.iloc[0,0]
        lastClose = testDf.iloc[0,2]
        lastBar = testDf.iloc[[0]]
        
        topHigh = None
        topLow = None
        
        counter = 1
        currentBar = testDf.iloc[counter]
        
        
        #check criteria
        while (counter < len(testDf) - 1):
            
            #original
            if (currentBar['Test: High'] > lastHigh
                and currentBar['Test: Low'] > lastLow
                and currentBar['Test: Close'] > self.testSMA[len(self.testSMA)-1]
                and lastClose < self.testSMA[len(self.testSMA)-2]):
                
                print("counter is at: ",counter)

                #we leave when we either have a 1% loss or 3% gain
                profitTarget = currentBar['Test: Close']*1.03
                stopLoss = currentBar['Test: Close']*0.99
                quantity = 50
                bracket = self.bracketOrder(self.testOrderId, 'BUY',quantity, profitTarget, stopLoss)
                contract = Contract()
                contract.symbol = self.symbol.upper() 
                contract.secType = 'STK'
                contract.exchange = 'SMART'
                contract.currency = 'USD'
                
                #place bracket order
                for x in bracket:
                    x.ocaGroup = 'OCA_'+str(self.testOrderId)
                    x.ocaType = 2
                    orderId = self.testOrderId
                    self.ib.placeOrder(x.orderId, contract, x)
                    
                self.testOrderId += 3


            counter += 1 
            currentBar = testDf.iloc[counter]
        
    # NOTE: to only be used outside of trading hours
    # should only be used for placing dummy trades
    def getHistoricalData(self):
        
        def run_loop():
            self.testIb.run()
        
        self.testIb = IBApi()
        self.testIb.connect('127.0.0.1', 7497, 90)

        #Start the socket in a thread
        api_thread = threading.Thread(target=run_loop, daemon=True)
        api_thread.start()

        time.sleep(1) 

        #Create contract object
        testContract = Contract()
        testContract.symbol = self.symbol
        testContract.secType = 'STK'
        testContract.exchange = 'SMART'
        testContract.currency = 'USD'
        testContract.primaryExchange = 'NASDAQ'
        
        #Request Trade Data
        self.testIb.reqHistoricalData(80, testContract,'', '1 D', '1 min', 'TRADES', 0, 1, False, [])

        recievedData = self.testIb.data
        
        time.sleep(5)
        self.testIb.disconnect()
        
        testDf = pd.DataFrame(recievedData, columns=['Date', 'Test: High', 'Test: Low', 'Test: Close',])
        testDf = testDf.set_index('Date')
        
        return testDf
        
    #bracket order
    def bracketOrder(self, parentOrderId, action, quantity, profitTarget, stopLoss):
        
        print(parentOrderId)
        
        # might be redundant, bring back if it breaks everything
        '''
        #initial entry
        contract = Contract()
        contract.symbol = self.symbol.upper() 
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        '''
        
        #create parent order 
        parent = Order()
        parent.orderId = parentOrderId+1
        parent.orderType = 'MKT'
        parent.action = action
        parent.totalQuantity = quantity
        parent.transmit = False 
        
        # make a target order
        profitOrder = Order()
        profitOrder.orderId = parent.orderId+2
        profitOrder.orderType = 'LMT'
        profitOrder.action = 'SELL'
        profitOrder.totalQuantity = quantity
        profitOrder.lmtPrice = round(profitTarget,3)
        profitOrder.transmit = False 
        
        # make a loss order
        stopOrder = Order()
        stopOrder.orderId = parent.orderId+3
        stopOrder.orderType = 'STP'
        stopOrder.action = 'SELL'
        stopOrder.totalQuantity = quantity
        stopOrder.parentId = parentOrderId 
        stopOrder.auxPrice = round(stopLoss,3)
        stopOrder.transmit = True 

        print("should be MKT: ",parent.orderType)
        print("should be LMT: ",profitOrder.orderType)
        print("should be STP: ",stopOrder.orderType)
        
        return [parent, profitOrder, stopOrder]
    
    '''
    TESTING FOR REALTIME DATA
    '''
    #Pass realtime bar data back to our bot object
    def on_bar_update(self, reqId, bar,realtime):
        global orderId
        
        #Historical Data to catch up
        if (realtime == False):
            self.bars.append(bar)
       
        else:

            bartime = datetime.strptime(bar.date,"%Y%m%d %H:%M:%S").astimezone(pytz.timezone("America/New_York"))
            
            # for the case of self.initialbartime
            # below code block is never even called, so that's why it is commented out
            '''
            temps = datetime.now().astimezone(pytz.timezone("America/New_York"))
            tempString = str(temp)
            size = len(tempString)
            mod_string = tempString[:size - 6]
            finaltemp = datetime.strptime(mod_string,"%Y-%m-%d %H:%M:%S.%f")
            '''
            
            minutes_diff = (bartime-self.initialbartime).total_seconds() / 60.0 #new 
            
            # get bar time w/o UTC 
            self.currentBar.date = bartime
            
            lastBar = self.bars[len(self.bars)-1]
            getBarsizeInt = int(self.barsize)
            
            #On Bar Close
            if (minutes_diff > 0 and math.floor(minutes_diff) % getBarsizeInt == 0):
                print("enter plz")
                print("orderId is: ",orderId)
                
                self.initialbartime = bartime 
                
                #Entry - If we have a higher high, a higher low and we cross the 50 SMA Buy
                #1.) SMA
                closes = []
                for bar in self.bars:
                    closes.append(bar.close)
                self.close_array = pd.Series(np.asarray(closes))
                
                self.sma = ta.trend.sma_indicator(self.close_array,self.smaPeriod,True)
                
                print("SMA : " + str(self.sma[len(self.sma)-1]))
                #2.) Calculate Higher Highs and Lows
                lastLow = self.bars[len(self.bars)-1].low
                lastHigh = self.bars[len(self.bars)-1].high
                lastClose = self.bars[len(self.bars)-1].close

                # Check Criteria
                # original
                '''
                if (bar.close > lastHigh
                    and self.currentBar.low > lastLow
                    and bar.close > str(self.sma[len(self.sma)-1])
                    and lastClose < str(self.sma[len(self.sma)-2])):
                '''
                # just for testing
                if (bar.close < self.sma[len(self.sma)-1]):
                
                    print("made it within criteria!")
                    #Bracket Order 2% Profit Target 1% Stop Loss
                    profitTarget = bar.close*1.05
                    stopLoss = bar.close*0.95
                    quantity = 3
                    
                    print("the orderId going into the bracket: ",orderId)
                    bracket = self.bracketOrder(orderId,"BUY",quantity, profitTarget, stopLoss)
                    
                    contract = Contract()
                    contract.symbol = self.symbol.upper()
                    contract.secType = "STK"
                    contract.exchange = "SMART"
                    contract.currency = "USD"
                    #Place Bracket Order
                    for o in bracket:
                        print("in bracket order")
                        o.ocaGroup = "OCA_"+str(orderId)
                        print("tell me what u are: ",o.action)
                        self.ib.placeOrder(o.orderId,contract,o) 
                        # potential solution for ib.sleep ->
                        # https://groups.io/g/insync/topic/just_parent_order_being/86518788?p=
                        # need a .sleep here, but should be ib.sleep and not time.sleep
                    orderId += 4
                #Bar closed append
                self.currentBar.close = bar.close
                print("New bar!")
                self.bars.append(self.currentBar)
                self.currentBar = Bar()
                self.currentBar.open = bar.open
       
        #Build  realtime bar
        if (self.currentBar.open == 0):
            self.currentBar.open = bar.open
        if (self.currentBar.high == 0 or bar.high > self.currentBar.high):
            self.currentBar.high = bar.high
        if (self.currentBar.low == 0 or bar.low < self.currentBar.low):
            self.currentBar.low = bar.low


In [5]:
####################### Start Bot #########################
bot = Bot()

TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm.nj
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:cashfarm
TWS Error Code:  2104
Message for Error Code:  Market data farm connection is OK:usfarm
TWS Error Code:  2106
Message for Error Code:  HMDS data farm connection is OK:euhmds
TWS Error Code:  2106
Message for Error Code:  HMDS data farm connection is OK:fundfarm
TWS Error Code:  2106
Message for Error Code:  HMDS data farm connection is OK:ushmds
TWS Error Code:  2158
Message for Error Code:  Sec-def data farm connection is OK:secdefil
Enter the ticker you want to trade: NIO
Enter the bar size: 1
Requesting market data...
1
Real-time bars during trading hours
strptime() argument 1 must be str, not int
Real-time bars during trading hours
strptime() argument 1 must be str, not int
Real-time bars during trading hours
strptime() argument 1 must be str, not int
Real-time bars during trading hours
strptime() argume